In [2667]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np

In [2668]:
def getYFinanceData(ticker, period, interval):
  dataF = yf.download(ticker, period=period, interval=interval)

  dataF.iloc[:,:]
  #dataF.Open.iloc

  df = pd.DataFrame(dataF)

  # use df index, convert DateTime to  a column instead of index
  df.reset_index(inplace=True)

  # delete Adj Close
  df = df.drop(['Adj Close'], axis=1)

  # rename Datetime to "Gmt time"
  df = df.rename(columns={'Datetime':'Gmt time'})
  
  # rename Date to "Gmt time"
  df = df.rename(columns={'Date':'Gmt time'})

  df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
  df.set_index("Gmt time", inplace=True)
  df=df[df.High!=df.Low]

  return df

In [2669]:
def calculateTechnicalAnalysis(df):
  df["MACD"]=ta.macd(df.Close)['MACD_12_26_9']
  df["MACD_HIST"]=ta.macd(df.Close)['MACDh_12_26_9']
  df["MACD_SIGNAL"]=ta.macd(df.Close)['MACDs_12_26_9']
  df["ADX"]=ta.adx(df.High, df.Low, df.Close)['ADX_14']
  df["20_MA"] = ta.ema(df.Close, length=20)
  df["50_MA"] = ta.ema(df.Close, length=50)
  df["100_MA"] = ta.ema(df.Close, length=100)
  df["200_MA"]= ta.ema(df.Close, length=200)
  # df["HA_OPEN"]=ta.ha(df.Open, df.High, df.Low, df.Close)['HA_open']
  # df["HA_CLOSE"]=ta.ha(df.Open, df.High, df.Low, df.Close)['HA_close']


  df['RSI']=ta.rsi(df.Close, length=16)
  df['MA_200']=ta.sma(df.Close, length=200)
  my_bbands = ta.bbands(df.Close, length=14, std=2.0)
  df=df.join(my_bbands)
  return df

In [2670]:
# df = getYFinanceData("GC=F", "730d", "60m")
# df1d = getYFinanceData("GC=F", "730d", "1d")
df = getYFinanceData("^NDX", "730d", "60m")
df1d = getYFinanceData("^NDX", "730d", "1d")
# df = getYFinanceData("^SPX", "730d", "60m")
# df1d = getYFinanceData("^SPX", "730d", "1d")
# df = getYFinanceData("MSFT", "730d", "60m")
# df1d = getYFinanceData("MSFT", "730d", "1d")
# df = getYFinanceData("AAPL", "730d", "60m")
# df1d = getYFinanceData("AAPL", "730d", "1d")
# df = getYFinanceData("NVDA", "730d", "60m")
# df1d = getYFinanceData("NVDA", "730d", "1d")
# df = getYFinanceData("TSLA", "730d", "60m")
# df1d = getYFinanceData("TSLA", "730d", "60m")
# df = getYFinanceData("1211.HK", "730d", "60m")
# df1d = getYFinanceData("1211.HK", "730d", "1d")



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2671]:
# def transformTo4H(df):
#     df4H = df.resample('4H').agg({
#         'Open': 'first',
#         'High': 'max',
#         'Low': 'min',
#         'Close': 'last',
#         'Volume': 'sum'
#     })
#     return df4H
# df = transformTo4H(df)

In [2672]:
df = calculateTechnicalAnalysis(df)
df1d = calculateTechnicalAnalysis(df1d)

In [2673]:
# create the column called Date. its value is the index without the time
df['Date'] = df.index.date
df1d['Date'] = df1d.index.date

In [2674]:
# for each row in df, get MACD, MACD_HIST, MACD_SIGNAL from df1d if the Date is the same as df1d's index which is also a date

# Ensure 'Date' in df and index in df1d are in the same datetime format
df['Date'] = pd.to_datetime(df['Date'])
df1d.index = pd.to_datetime(df1d.index)

# Use a try/except block to handle missing dates
def get_macd(row):
    try:
        return df1d.loc[row['Date']]['MACD']
    except KeyError:
        return np.nan
    
    
def get_macd_hist(row):
    try:
        return df1d.loc[row['Date']]['MACD_HIST']
    except KeyError:
        return np.nan
    
def get_macd_signal(row):
    try:
        return df1d.loc[row['Date']]['MACD_SIGNAL']
    except KeyError:
        return np.nan
    
def get_adx(row):
    try:
        return df1d.loc[row['Date']]['ADX']
    except KeyError:
        return np.nan
    


df['MACD_1d'] = df.apply(get_macd, axis=1)
df['MACD_HIST_1d'] = df.apply(get_macd_hist, axis=1)
df['MACD_SIGNAL_1d'] = df.apply(get_macd_signal, axis=1)
df['ADX_1d'] = df.apply(get_adx, axis=1)

In [2675]:
df

,Open,High,Low,Close,Volume,MACD,MACD_HIST,MACD_SIGNAL,ADX,20_MA,...,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0,Date,MACD_1d,MACD_HIST_1d,MACD_SIGNAL_1d,ADX_1d
Gmt time,,,,,,,,,,,,,,,,,,,,,
2021-04-30 09:30:00,13857.300781,13957.274414,13857.300781,13957.274414,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN,NaN
2021-04-30 10:30:00,13959.013672,13969.001953,13877.937500,13882.177734,78448743,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN,NaN
2021-04-30 11:30:00,13880.872070,13929.281250,13876.458008,13912.838867,53172607,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN,NaN
2021-04-30 12:30:00,13913.563477,13919.535156,13886.602539,13907.411133,53841565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN,NaN
2021-04-30 13:30:00,13907.675781,13910.697266,13841.281250,13873.360352,63393587,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-25 10:30:00,18283.859375,18307.992188,18270.974609,18296.777344,100800785,59.562507,-16.189681,75.752188,23.051175,18281.230790,...,18248.886431,18346.116350,18443.346270,1.059951,0.246277,2024-03-25,149.392621,4.556348,144.836274,16.328932
2024-03-25 11:30:00,18297.582031,18318.261719,18287.408203,18293.130859,748943521,53.491692,-17.808397,71.300089,21.721180,18282.364130,...,18257.894041,18334.012416,18410.130792,0.830352,0.231461,2024-03-25,149.392621,4.556348,144.836274,16.328932
2024-03-25 12:30:00,18293.324219,18314.716797,18283.644531,18312.734375,485000717,49.689576,-17.288410,66.977986,20.541012,18285.256534,...,18268.315363,18326.031948,18383.748532,0.629886,0.384803,2024-03-25,149.392621,4.556348,144.836274,16.328932


In [2676]:
# VWAPsignal = [0]*len(df)
# backcandles = 15

# for row in range(backcandles, len(df)):
#     upt = 1
#     dnt = 1
#     for i in range(row-backcandles, row+1):
#         if max(df.Open[i], df.Close[i])>=df.VWAP[i]:
#             dnt=0
#         if min(df.Open[i], df.Close[i])<=df.VWAP[i]:
#             upt=0
#     if upt==1 and dnt==1:
#         VWAPsignal[row]=3
#     elif upt==1:
#         VWAPsignal[row]=2
#     elif dnt==1:
#         VWAPsignal[row]=1

# df['VWAPSignal'] = VWAPsignal

### MACD Signal

In [2677]:
backcandles = 15
# calcualte macd signal
macd_signal = [0]*len(df)
for row in range(1, len(df)):
    if(
        True
        # (df.MACD_SIGNAL[row] < df.MACD[row]
        #     and df.MACD_SIGNAL[row-1] > df.MACD[row-1]
        #     and df.MACD_HIST[row] > 0)
        # or (df.MACD[row] > df.MACD[row-1]
        #     and df.MACD[row-1] < 0
        #     and df.MACD[row] > 0
        #     )
        # df.MACD_SIGNAL[row-1] < 0  
        # and df.MACD[row] < 0
        # and df.ADX[row] > 25
        # and df.Close[row] > df['200_MA'][row]
        # and df['20_MA'][row] > df['200_MA'][row]
        # and df["HA_CLOSE"][row] - df["HA_OPEN"][row] > df["HA_CLOSE"][row - 1] - df["HA_OPEN"][row - 1]
        # and df["HA_CLOSE"][row] > df["HA_OPEN"][row]
        and df.MACD_HIST[row-1] < 0 and df.MACD_HIST[row] > 0
        # and df.MACD_HIST_1d[row-1] < 0 and df.MACD_HIST_1d[row] > 0
        # and df.MACD_HIST_1d[row] > 0
        # and df.MACD_SIGNAL_1d[row] > df.MACD_1d[row]
        # and df.MACD_HIST_1d[row] > 0
        # and df.Close[row] > df["20_MA"][row]
    ):
        macd_signal[row]=1
    elif  (
        True
        # (df.MACD_SIGNAL[row] > df.MACD[row]
        #     and df.MACD_SIGNAL[row-1] < df.MACD[row-1]
        #     and df.MACD_HIST[row] < 0)  
        # or (df.MACD[row] < df.MACD[row-1]
        #     and df.MACD[row-1] > 0
        #     and df.MACD[row] < 0
        #     )
        # df.MACD_SIGNAL[row-1] > 0  
        # and df.MACD[row] > 0
        # and df.ADX[row] > 25
        # and df.Close[row] < df['200_MA'][row]
        # and df['20_MA'][row] < df['200_MA'][row]
        # and df["HA_CLOSE"][row] - df["HA_OPEN"][row] < df["HA_CLOSE"][row - 1] - df["HA_OPEN"][row - 1]
        # and df["HA_CLOSE"][row] < df["HA_OPEN"][row]
        and df.MACD_HIST[row-1] > 0 and df.MACD_HIST[row] < 0
        # and df.MACD_HIST_1d[row-1] > 0 and df.MACD_HIST_1d[row] < 0
        # and df.MACD_HIST_1d[row] < 0
        # and df.MACD_SIGNAL_1d[row] < df.MACD_1d[row]
        # and df.MACD_HIST_1d[row] < 0
        # and df.Close[row] < df["20_MA"][row]
    ):
        macd_signal[row]=-1

df['MACDSignal'] = macd_signal


/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1398393693.py:21: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1398393693.py:45: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



### RSI Signal

In [2678]:
import numpy as np

backcandles = 14

# calcualte macd signal
rsi_signal = [0]*len(df)
df.RSI_14MA = df.RSI
for row in range(1, len(df)):
    if row < backcandles:
        continue
    previous_rsi = df.RSI[row-backcandles:row]
    previous_rsi_signal = df.RSI[row-backcandles:row]
    rsi_slope = np.polyfit(range(0, len(previous_rsi)), previous_rsi, 1)[0]


    if(
        # RSI is increasing for the last backcandles
        True
        # and rsi_slope > 0
        and df.RSI[row] > 50
        or (df.RSI[row]<25 and df.RSI[row-1]>25)
        # and df.RSI[row] > 50
        # (df.RSI[row]>30 and df.RSI[row-1]<30)

    ):
        rsi_signal[row]=1
    elif  (
        True
        # rsi is decreasing
        # and rsi_slope < 0
        and df.RSI[row] < 55
        or (df.RSI[row]>75 and df.RSI[row-1]<75)
        # and df.RSI[row] < 50
        # (df.RSI[row]<70 and df.RSI[row-1]>70)
    ):
        rsi_signal[row]=-1

df['RSI_Signal'] = rsi_signal

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/3475201347.py:7: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/3475201347.py:20: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/3475201347.py:21: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/3475201347.py:31: FutureWarning:

Series.__getitem__ treat

### MA Signal

In [2679]:
import numpy as np

backcandles = 10

# calcualte macd signal
ma_signal = [0]*len(df)
df.MA_SIGNAL = df['200_MA']
for row in range(1, len(df)):
    if row < backcandles:
        continue
    previous_ma = df['100_MA'][row-backcandles:row]
    previous_ma_signal = df['100_MA'][row-backcandles:row]
    ma_slope = np.polyfit(range(0, len(previous_ma)), previous_ma, 1)[0]


    if(
        True
        and df['Close'][row] > df['50_MA'][row]
        # and df['20_MA'][row] > df['50_MA'][row]
        # ma_slope > 0
    ):
        ma_signal[row]=1
    elif  (
        True
        and df['Close'][row] < df['50_MA'][row]
        # and df['20_MA'][row] < df['50_MA'][row]
        # ma_slope < 0
    ):
        ma_signal[row]=-1

df['MA_Signal'] = ma_signal

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1000703136.py:7: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1000703136.py:18: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1000703136.py:25: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



### Total Signal

In [2680]:
def TotalSignal(l):
    # if (df.VWAPSignal[l]==2
    #     and df.Close[l]<=df['BBL.0'][l]
    #     and df.RSI[l]<45):
    #         return 2
    # if (df.VWAPSignal[l]==1
    #     and df.Close[l]>=df['BBU.0'][l]
    #     and df.RSI[l]>55):
    #         return 1
    # return 0

    if (
        df.MACDSignal[l]==1
        and df.RSI_Signal[l]==1
        # and df.MA_SIGNAL[l]==1
        # and df.ADX_1d[row] > 5
        # and (df.RSI[l]<30)
        # and (df.RSI[l]>50)
    ):
            return 2
    if (
        df.MACDSignal[l]==-1
        and df.RSI_Signal[l]==-1
        # and df.MA_Signal[l]==-1
        # and df.ADX_1d[row] > 5
        # and (df.RSI[l]>70)
        # and (df.RSI[l]<50)
    ):
            return 1
    return 0
        
TotSignal = [0]*len(df)
for row in range(backcandles, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1663792097.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1663792097.py:22: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1663792097.py:23: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipy

In [2681]:
df[df.TotalSignal!=0].count()

Open              225
High              225
Low               225
Close             225
Volume            225
MACD              225
MACD_HIST         225
MACD_SIGNAL       225
ADX               225
20_MA             225
50_MA             224
100_MA            222
200_MA            219
RSI               225
MA_200            219
BBL_14_2.0        225
BBM_14_2.0        225
BBU_14_2.0        225
BBB_14_2.0        225
BBP_14_2.0        225
Date              225
MACD_1d           220
MACD_HIST_1d      218
MACD_SIGNAL_1d    218
ADX_1d            220
MACDSignal        225
RSI_Signal        225
MA_Signal         225
TotalSignal       225
dtype: int64

In [2682]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [2683]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=0
dfpl = df[st:st+ len(df)]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                # go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                #            line=dict(color='blue', width=1), 
                #            name="VWAP"), 
                # go.Scatter(x=dfpl.index, y=dfpl['BBL.0'], 
                #            line=dict(color='green', width=1), 
                #            name="BBL"),
                # go.Scatter(x=dfpl.index, y=dfpl['BBU.0'], 
                #            line=dict(color='green', width=1), 
                #            name="BBU")
                go.Scatter(x=dfpl.index, y=dfpl['200_MA'],
                            line=dict(color='red', width=1), 
                            name="200_MA"),
      ])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")

# set figure size
fig.update_layout(
    autosize=False,
    width=1600,
    height=1000,
    margin=dict(l=50,r=50,b=100,t=100,pad=4)
)

fig.show()

In [2684]:
dfpl = df.copy()
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=1)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [2685]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.03
    mysize = initsize
    latestEntry = 0
    lastHigh = 0
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 9*self.data.ATR[-1]
        TPSLRatio = 1.5
        
        # if the current equity is higher than lastHigh, then update lastHigh
        if (self.equity > self.lastHigh):
            self.lastHigh = self.equity
            
        # if self.position.pl is down for more than 2 percent of the height, close all positions
        if (self.position.pl < -1 * self.lastHigh * 0.02):
            self.position.close()
            print("Close position due to loss")
            print("------")
        
        # # close the position if the amount lost exceeds 3% of the account balance
        if (self.position.pl < -1 * self.equity * 0.02):
            self.position.close()
            print("Close position due to loss")
            print("------")

        if len(self.trades)>0:
            for trade in self.trades:
                if trade.is_long and self.data.RSI[-1]>=90:
                    trade.close()
                elif trade.is_short and self.data.RSI[-1]<=10:
                    trade.close()
                
                # update the stop loss to entry if the pl is greater than the stop loss amount
                # if (trade.pl > self.equity * 0.01):
                #     trade.sl = trade.entry_price
                #     print("Update stop loss to entry")
                #     print("------")
                
                # close the position if the amount lost exceeds 3% of the account balance
                # if (trade.pl < -1 * self.equity * 0.01):
                #     trade.close()
                #     print("Close position due to loss")
                #     print("------")
                
        # # # if the pl is greater than the stop loss amount, then move the stop loss to entry
        # if (self.position.pl > self.latestEntry * 0.02):
        #     # adjust the stop loss to entry
        #     self.position.sl = self.latestEntry
        #     # # for each opened position, update the stop loss to the entry
        #     # for trade in self.trades:
        #     #     trade.sl = self.latestEntry
                
                
        
        # print("Datetime: ", self.data.index[-1])
        if self.signal1==2 and len(self.trades) < 3:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.latestEntry = self.data.Close[-1]
            print("Buy", self.data.Close[-1])
            print("Stop loss: ", sl1)
            print("Take profit: ", tp1)
            print("------")
        
        elif self.signal1==1 and len(self.trades) < 1:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.latestEntry = self.data.Close[-1]
            print("Sell", self.data.Close[-1])
            print("Stop loss: ", sl1)
            print("Take profit: ", tp1)
            print("------")

        # else:
        #     print("No signal")

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/100, commission=0)
stat = bt.run()

Sell 13530.32421875
Stop loss:  14055.9892578125
Take profit:  12741.82666015625
------
Buy 13423.857421875
Stop loss:  11769.4921875
Take profit:  15905.4052734375
------
Buy 13634.244140625
Stop loss:  11534.37890625
Take profit:  16784.0419921875
------
Close position due to loss
------
Close position due to loss
------
Buy 13724.701171875
Stop loss:  11781.0
Take profit:  16640.2529296875
------
Buy 13932.251953125
Stop loss:  12612.3896484375
Take profit:  15912.04541015625
------
Buy 14005.896484375
Stop loss:  13302.7978515625
Take profit:  15060.54443359375
------
Buy 15072.58203125
Stop loss:  14391.5
Take profit:  16094.205078125
------
Buy 14758.3212890625
Stop loss:  14214.97265625
Take profit:  15573.34423828125
------
Buy 15631.24609375
Stop loss:  14734.708984375
Take profit:  16976.0517578125
------
Buy 15959.1435546875
Stop loss:  15049.625
Take profit:  17323.42138671875
------
Buy 16357.9423828125
Stop loss:  15620.8828125
Take profit:  17463.53173828125
------
Close

In [2686]:
print(stat)

Start                     2021-04-30 09:30:00
End                       2024-03-25 14:30:00
Duration                   1060 days 05:00:00
Exposure Time [%]                   77.349587
Equity Final [$]               1701710.500977
Equity Peak [$]                1750078.258789
Return [%]                        1601.710501
Buy & Hold Return [%]               31.270565
Return (Ann.) [%]                  166.373702
Volatility (Ann.) [%]              244.905113
Sharpe Ratio                         0.679339
Sortino Ratio                        3.474715
Calmar Ratio                         3.553907
Max. Drawdown [%]                  -46.814307
Avg. Drawdown [%]                   -5.892652
Max. Drawdown Duration      262 days 00:00:00
Avg. Drawdown Duration        9 days 06:00:00
# Trades                                  105
Win Rate [%]                        38.095238
Best Trade [%]                       21.26462
Worst Trade [%]                      -5.53113
Avg. Trade [%]                    

In [2687]:
bt.plot(show_legend=False)

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_3406/1243017189.py:1: UserWarning:

Superimposed OHLC plot matches the original plot. Skipping.



Row(id='392312', ...)